#### This notebook explains how to uncover risks related to your usecase based on a given taxonomy.


##### Import libraries


In [1]:
from ai_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from ai_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)
from ai_atlas_nexus.library import AIAtlasNexus

/Users/dhaval/Projects/Usage-Governance/ai-atlas-nexus/src/ai_atlas_nexus/blocks/risk_explorer/explorer.py:8: UserWarning: Deprecated, Please use AtlasExplorer instead
  warnings.warn("Deprecated, Please use AtlasExplorer instead")
/Users/dhaval/Projects/Usage-Governance/ai-atlas-nexus/src/ai_atlas_nexus/toolkit/job_utils.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


##### AI Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/ai-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [2]:
# inference_engine = OllamaInferenceEngine(
#     model_name_or_path="granite3.3:8b",
#     credentials=InferenceEngineCredentials(api_url="OLLAMA_API_URL"),
#     parameters=OllamaInferenceEngineParams(
#         num_predict=1000, num_ctx=8192, temperature=0, repeat_penalty=1
#     ),
# )

# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-3-3-8b-instruct",
#     credentials={
#         "api_key": "WML_API_KEY",
#         "api_url": "WML_API_URL",
#         "project_id": "WML_PROJECT_ID",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=1000, decoding_method="greedy", repetition_penalty=1
#     ),
# )

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.3-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="VLLM_API_URL", api_key="VLLM_API_KEY"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=1000, temperature=0),
# )

inference_engine = RITSInferenceEngine(
    model_name_or_path="ibm-granite/granite-3.3-8b-instruct",
    credentials={
        "api_key": "cbc683b3a1a7c52d2a73008b785d2811",
        "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
    },
    parameters=RITSInferenceEngineParams(
        max_completion_tokens=1000, temperature=0, logprobs=True, top_logprobs=3
    ),
)

[2025-12-16 12:24:54:241] - INFO - AIAtlasNexus - Created RITS inference engine.


##### Create an instance of AIAtlasNexus

_Note: (Optional)_ You can specify your own directory in `AIAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [3]:
ai_atlas_nexus = AIAtlasNexus()

[2025-12-16 12:24:54:504] - INFO - AIAtlasNexus - Created AIAtlasNexus instance. Base_dir: None


##### Risk Identification API

AIAtlasNexus.identify_risks_from_usecases()

Params:

- usecases (List[str]):
  A List of strings describing AI usecases
- inference_engine (InferenceEngine):
  An LLM inference engine to infer risks from the usecases.
- taxonomy (str, optional):
  The string label for a taxonomy. Default to None.
- cot_examples (Dict[str, List], optional):
  The Chain of Thought (CoT) examples to use in the risk identification.
  The example template is available at src/ai_atlas_nexus/data/templates/risk_generation_cot.json.
  Assign the ID of the taxonomy you wish to use as the key for CoT examples. Providing this value
  will override the CoT examples present in the template master. Default to None.
- max_risk (int, optional):
  The maximum number of risks to extract. Pass None to allow the inference engine to determine the number of risks. Defaults to None.
- zero_shot_only (bool): If enabled, this flag allows the system to perform Zero Shot Risk identification, and the field `cot_examples` will be ignored.


##### Risk Identification using IBM AI Risk taxonomy


In [4]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

risks = ai_atlas_nexus.identify_risks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
    taxonomy="ibm-risk-atlas",
    max_risk=5,
)

print(len(risks[0]))
for risk in risks[0]:
    print(risk.name)

Inferring with RITS:   0%|          | 0/99 [00:00<?, ?it/s]

56
Over- or under-reliance
Confidential data in prompt
Data privacy rights alignment
Discriminatory actions
Legal accountability
Hallucination
Mitigation and maintenance
AI agent compliance
Function calling hallucination
Confidential information in data
Lack of model transparency
Unrepresentative data
AI agents' impact on human agency
Personal information in prompt
Sharing IP/PI/confidential information with user
Lack of testing diversity
Decision bias
Exposing personal information
Improper data curation
Over- or under-reliance on AI agents
Revealing confidential information
Uncertain data provenance
Data bias
Unauthorized use
Misaligned actions
Incomplete usage definition
Lack of data transparency
Impact on affected communities
Improper retraining
Introduce data bias
Accountability of AI agent actions
Incomplete AI agent evaluation
Inaccessible training data
Non-disclosure
Lack of training data transparency
Reproducibility
Incomplete advice
Prompt injection attack
Lack of system trans

##### Risk Identification using IBM AI Risk taxonomy with Custom CoT examples

**Note:** To use custom risk `cot_examples` for a new taxonomy or an existing taxonomy, users must provide their own set of example usecase and associated risks in a JSON file such as in [risk_generation_cot.json](https://github.com/IBM/ai-atlas-nexus/blob/main/src/ai_atlas_nexus/data/templates/risk_generation_cot.json). This will enable the LLM to learn from these few-shot examples and generate better responses. Please follow the guide [here](https://ibm.github.io/ai-atlas-nexus/concepts/Chain_of_thought_templates/#risk-identification).


In [5]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

risks = ai_atlas_nexus.identify_risks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
    taxonomy="ibm-risk-atlas",
    cot_examples={
        "ibm-risk-atlas": [
            {
                "Usecase": "In a medical chatbot, generative AI can be employed to create a triage system that assesses patients' symptoms and provides immediate, contextually relevant advice based on their medical history and current condition. The chatbot can analyze the patient's input, identify potential medical issues, and offer tailored recommendations or insights to the patient or healthcare provider. This can help streamline the triage process, ensuring that patients receive the appropriate level of care and attention, and ultimately improving patient outcomes.",
                "Risks": [
                    "Improper usage",
                    "Incomplete advice",
                    "Lack of model transparency",
                    "Lack of system transparency",
                    "Lack of training data transparency",
                    "Data bias",
                    "Uncertain data provenance",
                    "Lack of data transparency",
                    "Impact on human agency",
                    "Impact on affected communities",
                    "Improper retraining",
                    "Inaccessible training data",
                ],
            }
        ]
    },
    max_risk=5,
)

risks

Inferring with RITS:   0%|          | 0/99 [00:00<?, ?it/s]

[[Risk(id='atlas-over-or-under-reliance', name='Over- or under-reliance', description='In AI-assisted decision-making tasks, reliance measures how much a person trusts (and potentially acts on) a model’s output. Over-reliance occurs when a person puts too much trust in a model, accepting a model’s output when the model’s output is likely incorrect. Under-reliance is the opposite, where the person doesn’t trust the model but should.', url='https://www.ibm.com/docs/en/watsonx/saas?topic=SSYOK8/wsj/ai-risk-atlas/over-or-under-reliance.html', dateCreated=datetime.date(2024, 3, 6), dateModified=datetime.date(2025, 10, 22), isDetectedBy=[], hasRelatedAction=[], isDefinedByTaxonomy='ibm-risk-atlas', isPartOf='ibm-risk-atlas-value-alignment', closeMatch=['credo-risk-016'], exactMatch=[], broadMatch=['nist-human-ai-configuration'], narrowMatch=[], relatedMatch=['llm052025-improper-output-handling', 'llm062025-excessive-agency', 'mit-ai-causal-risk-entity-human', 'mit-ai-causal-risk-intent-unint